In [1]:
import os
import pandas as pd

In [4]:
input_directories = [
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/ad_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/dlb_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/mci_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/nph_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/vad_vs_nc",
]

# input_directories = [
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/vad_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/vad_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/nph_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/nph_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/mci_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/mci_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/dlb_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/dlb_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/ad_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/ad_nc_female", 
# ]

Welch_ANOVA

In [6]:
def filter_significant_features(directory):
    # List only CSV files containing '_welch' in the filename
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv') and '_welch' in f]
    
    if len(files) != 3:
        raise ValueError("Each directory must contain exactly three Welch ANOVA CSV files.")

    significant_features_sets = []
    total_features = 0
    
    for file in files:
        data = pd.read_csv(file)
        total_features += len(data)  # Sum up all features before filtering
        significant_features = data[data['P-Value'] < 0.03]['Feature']
        significant_features_sets.append(set(significant_features))

    # Intersection of significant features across all files
    common_features = set.intersection(*significant_features_sets)
    return common_features, total_features

welch_anova_common_features_dict = {}
total_initial_features_dict = {}

# input_directories = [
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/vad_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/vad_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/nph_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/nph_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/mci_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/mci_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/dlb_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/dlb_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/ad_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/ad_nc_female", 
# ]

input_directories = [
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/ad_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/dlb_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/mci_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/nph_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/vad_vs_nc",
]
for directory in input_directories:
    dataset_type = directory.split('/')[-1]
    try:
        common_features, total_features = filter_significant_features(directory)
        welch_anova_common_features_dict[dataset_type] = common_features
        total_initial_features_dict[dataset_type] = total_features
        filtered_out_features_count = total_features - len(common_features)
        print(f"{dataset_type}: Initially {total_features} features, {filtered_out_features_count} features filtered out, leaving {len(common_features)} significant features.")
    except ValueError as e:
        print(f"Error for {dataset_type}: {e}")

ad_vs_nc: Initially 7686 features, 7009 features filtered out, leaving 677 significant features.
dlb_vs_nc: Initially 7686 features, 7094 features filtered out, leaving 592 significant features.
mci_vs_nc: Initially 7686 features, 7647 features filtered out, leaving 39 significant features.
nph_vs_nc: Initially 7686 features, 7382 features filtered out, leaving 304 significant features.
vad_vs_nc: Initially 7686 features, 7139 features filtered out, leaving 547 significant features.


In [41]:
# Optionally print the dictionaries if needed to see the detailed output
# print("Common Features:", welch_anova_common_features_dict)
# print("Total Initial Features:", total_initial_features_dict)

RELIEFF

In [42]:
import os
import pandas as pd

In [7]:
def filter_top_relieff_features(directory):
    # List only CSV files containing '_relieff' in the filename
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv') and '_relieff' in f]
    
    if len(files) != 3:
        raise ValueError("Each directory must contain exactly three ReliefF CSV files.")

    top_features_sets = []
    total_features = 0
    
    for file in files:
        data = pd.read_csv(file)
        total_features += len(data)  # Sum up all features before filtering
        top_count = int(len(data) * 0.05)  # Top 10%
        top_features = set(data.nlargest(top_count, 'ReliefF Score')['Feature'])
        top_features_sets.append(top_features)

    # Intersection of top features across all files
    common_features = set.intersection(*top_features_sets)
    return common_features, total_features

relieff_common_features_dict = {}
total_initial_features_dict = {}

# Example input directories (update paths as necessary)
# input_directories = [
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/vad_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/vad_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/nph_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/nph_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/mci_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/mci_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/dlb_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/dlb_nc_female",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/ad_nc_male",
#     "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_sex_specific/ad_nc_female", 
# ]

input_directories = [
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/ad_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/dlb_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/mci_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/nph_vs_nc",
    "/home/aghasemi/CompBio481/feature_selection/feat_select_res_branch1_overall/vad_vs_nc",
]
for directory in input_directories:
    dataset_type = directory.split('/')[-1]
    try:
        common_features, total_features = filter_top_relieff_features(directory)
        relieff_common_features_dict[dataset_type] = common_features
        total_initial_features_dict[dataset_type] = total_features
        filtered_out_features_count = total_features - len(common_features)
        print(f"{dataset_type}: Initially {total_features} features, {filtered_out_features_count} features filtered out, leaving {len(common_features)} significant features.")
    except ValueError as e:
        print(f"Error for {dataset_type}: {e}")

ad_vs_nc: Initially 7686 features, 7647 features filtered out, leaving 39 significant features.
dlb_vs_nc: Initially 7686 features, 7665 features filtered out, leaving 21 significant features.
mci_vs_nc: Initially 7686 features, 7671 features filtered out, leaving 15 significant features.
nph_vs_nc: Initially 7686 features, 7659 features filtered out, leaving 27 significant features.
vad_vs_nc: Initially 7686 features, 7639 features filtered out, leaving 47 significant features.


Combine Dictionaries

In [8]:
def combine_feature_dictionaries(dict1, dict2):
    # Combine two dictionaries where values are sets of features
    combined_dict = {}
    for key in dict1:
        if key in dict2:
            combined_dict[key] = dict1[key].union(dict2[key])
        else:
            combined_dict[key] = dict1[key]  # Only dict1 has this key
    for key in dict2:
        if key not in dict1:
            combined_dict[key] = dict2[key]  # Only dict2 has this key
    return combined_dict

# Assuming welch_anova_common_features_dict and relieff_common_features_dict are already defined
combined_features_dict = combine_feature_dictionaries(welch_anova_common_features_dict, relieff_common_features_dict)

# Print the combined results for each dataset condition
for condition, features in combined_features_dict.items():
    print(f"{condition}: {len(features)} significant features combined")

ad_vs_nc: 707 significant features combined
dlb_vs_nc: 607 significant features combined
mci_vs_nc: 54 significant features combined
nph_vs_nc: 331 significant features combined
vad_vs_nc: 594 significant features combined


Filter Datasets

In [9]:
# datasets = [
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/vad_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/vad_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/nph_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/nph_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/mci_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/mci_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/dlb_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/dlb_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/ad_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/ad_nc_female.csv",
# ]

datasets = [
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/ad_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/dlb_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/mci_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/nph_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/vad_vs_nc.csv",
]

In [10]:
def filter_datasets(datasets, features_dict, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Fixed columns to retain in each dataset
    fixed_columns = ['ID_1', 'Diagnosis', 'Age', 'Sex', 'APOE4']

    for dataset_path in datasets:
        # Extract the condition name from the file path
        condition = os.path.basename(dataset_path).replace('.csv', '')
        if condition in features_dict:
            # Load the dataset
            df = pd.read_csv(dataset_path)

            # Combine significant features with fixed columns
            significant_features = features_dict[condition]
            columns_to_keep = fixed_columns + list(significant_features)

            # Filter the DataFrame to include only the required columns
            filtered_df = df.loc[:, df.columns.isin(columns_to_keep)]

            # Reorder columns to ensure ID_1 is the first column
            columns_order = ['ID_1'] + [col for col in filtered_df.columns if col != 'ID_1']
            filtered_df = filtered_df[columns_order]

            # Prepare output file path
            output_file_path = os.path.join(output_directory, os.path.basename(dataset_path))

            # Save the filtered DataFrame
            filtered_df.to_csv(output_file_path, index=False)

            print(f"Filtered dataset saved to {output_file_path}")
        else:
            print(f"No features found for {condition}, skipping...")

In [11]:
# Define the datasets paths
datasets = [
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/ad_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/dlb_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/mci_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/nph_vs_nc.csv",
    "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/vad_vs_nc.csv",
]

# datasets = [
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/vad_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/vad_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/nph_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/nph_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/mci_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/mci_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/dlb_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/dlb_nc_female.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/ad_nc_male.csv",
#     "/home/aghasemi/CompBio481/datasets/processed_datasets/usable_datasets_branch1/ad_nc_female.csv",
# ]

In [12]:
# Define the output directory
output_directory = "/home/aghasemi/CompBio481/datasets/filtered_datasets_after_rank_feat_select_branch1_overall"

In [13]:
# Call the function
filter_datasets(datasets, combined_features_dict, output_directory)

Filtered dataset saved to /home/aghasemi/CompBio481/datasets/filtered_datasets_after_rank_feat_select_branch1_overall/ad_vs_nc.csv
Filtered dataset saved to /home/aghasemi/CompBio481/datasets/filtered_datasets_after_rank_feat_select_branch1_overall/dlb_vs_nc.csv
Filtered dataset saved to /home/aghasemi/CompBio481/datasets/filtered_datasets_after_rank_feat_select_branch1_overall/mci_vs_nc.csv
Filtered dataset saved to /home/aghasemi/CompBio481/datasets/filtered_datasets_after_rank_feat_select_branch1_overall/nph_vs_nc.csv
Filtered dataset saved to /home/aghasemi/CompBio481/datasets/filtered_datasets_after_rank_feat_select_branch1_overall/vad_vs_nc.csv
